<a href="https://colab.research.google.com/github/Charee-Villapong/charee-villapong/blob/master/get_tweet_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# 必要なモジュールのimport
import datetime
import pandas as pd
import tweepy
!pip install janome

In [32]:

# 各種ツイッターのキーをセット　consumer_key, consumer_secret, access_key, access_secret
consumer_key = "8ljKZRP1ekS9dzkhuaUr6mL8J"
consumer_secret = "fcpAsRKs4wg2jnUc98jJ1YG4kDLcumehO76T2CAQXH74gPeHjs"
access_key = "1289362986210009089-LYBDoTh8ipkpbCnOfy0DpaIfqQM7ZM"
access_secret = "loSkIIcmgZ7YmIx3y4IBqfTYIfN8aYc3Jva41SIFSmflP"

# 認証のためのAPIキーをセット
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_key,access_secret)
api = tweepy.API(auth)
#CjtzvLBhCeBmksgJE5B1qdTzo


# API利用制限にかかった場合、解除まで待機する

def main():
    '''
    メインの実行部分
    ツイートデータの取得からデータの出力まで
    '''

    # ツイートデータの取得 日付の指定は 2020-7-30のみでもOK,
    # 日本時間で取得したい場合は2020-7-30_00:00:00_JSTのように指定
    # JSTをつけないと時間がUTCになる UTCは協定世界時間-> JST＋9:00(日本時間よりも9時間進んでいる)
    tweet_data = get_serchtweet('Python','2021-01-23,00:00:00_JST','2021-01-30,00:00:00_JST',100,10)
    # ツイートデータを番号順に出力
    output_tweets(tweet_data)
    # ツイートデータをDataframeにする
    df = make_df(tweet_data)
    negapogi(df)
    # ツイートデータのCSVへの出力
    df.to_csv('tweet_data.csv')

# ツイートを収集する関数
def get_serchtweet(s,since,until,count,rlcount):
    '''
    ツイート情報を特定のキーワードで、期間を指定して収集
    取得できるデータは1週間以内のデータだけ
    リツイート数＋いいね数の合計でツイートを絞り込める
    '''

    # 検索キーワードの設定、 リツイートは除く
    search_key = s + "-filter:retweets"
    # ツイートデータ取得部分
    # tweepy.CursorのAPIのキーワードサーチを使用(api.search)
    # qがキーワード, sinceがいつから, untilがいつまで, tweet_modeでつぶやきの省略ありなし, langで言語, .items(数)と書いてツイート数を指定
    tweets = tweepy.Cursor(api.search,q=search_key,since = since ,until = until,tweet_mode = "extended", lang="ja").items(count)
    # ツイートのデータを取り出して、リストにまとめていく部分
    
    # ツイートデータを入れる空のリストを用意
    tweet_data = []
    for tweet in tweets:
        if tweet.retweet_count + tweet.favorite_count >= rlcount:# いいねとリツイートの合計がrlcuont以上の条件
            tweet_data.append([tweet.user.name,tweet.user.screen_name,tweet.retweet_count,tweet.favorite_count,tweet.created_at.strftime("%Y-%m-%d-%H:%M-%S"),tweet.full_text.replace("\n","")])# 空のリストにユーザーネーム、スクリーンネーム、RT数、いいね数、日付などを入れる
    return tweet_data

def output_tweets(tweet_data):
    '''
    ツイートのリストを順番をつけて出力する関数の作成
    '''
    for i in range(len(tweet_data)):
        print(f"({i}番目のつぶやき{tweet_data[i]}")


def make_df(tweet_data):
    '''
    ツイートデータからDataFrameを作成する
    '''

    # データを入れる空のリストを用意(ユーザー名、ユーザーid、RT数、いいね数、日付け、ツイート本文)
    list_username = []
    list_userid = []
    list_rtcount = []
    list_favorite = []
    list_date = []
    list_text = []

    # ツイートデータからユーザー名、ユーザーid、RT数、いいね数、日付け、ツイート本文のそれぞれをデータごとにまとめたリストを作る
    for i in range(len(tweet_data)):
        list_username.append(tweet_data[i][0])
        list_userid.append(tweet_data[i][1])
        list_rtcount.append(tweet_data[i][2])
        list_favorite.append(tweet_data[i][3])
        list_date.append(tweet_data[i][4])
        list_text.append(tweet_data[i][5])

    # 先ほど作ったデータごとにまとめたリストからDataframeの作成
    df = pd.DataFrame({'ユーザーID':list_userid,
                'ユーザー名':list_username,
                'RT数':list_rtcount,
                'いいね数':list_favorite,
                '日時':list_date,
                '本文':list_text})

    return df


In [35]:
# 形態素解析をするためのjanomeをインストール
from janome.tokenizer import Tokenizer

# pandas
import pandas as pd

def negapogi(df):
    '''
    データフレームを引数に受け取り、
    ネガポジ分析をする関数
    '''
    # 極性辞書をPythonの辞書にしていく
    np_dic = {}
    
    with open("/content/drive/MyDrive/pn.csv.m3.120408.trim", "r", encoding="utf-8") as f:  # 日本語評価極性辞書のファイルの読み込み
        lines = [line.replace('\n', '').split('\t') for line in f.readlines()] # 1行1行を読み込み、文字列からリスト化。リストの内包表記の形に

    posi_nega_df = pd.DataFrame(lines, columns = ['word', 'score', 'explain'])  # リストからデータフレームの作成
    
    # データフレームの2つの列から辞書の作成　zip関数を使う
    np_dic = dict(zip(posi_nega_df.word, posi_nega_df.score))

    # 形態素解析をするために必要な記述を書いていく
    tokenizer = Tokenizer()

    # ツイート一つ一つを入れてあるデータフレームの列（本文の列）をsentensesと置く
    sentences = df['本文']
    # p,n,e,?p?nを数えるための辞書を作成
    result = {'p':0,'n':0,'e':0,'?p?n':0}
    
    # ツイートを一つ一つ取り出す
    # 形態素解析をする部分
    # ツイートに含まれる単語を抜き出す
    # 辞書のキーとして単語があるかどうかの存在確認
    # 値(pかnかeか?p?nのどれか)をvalueという文字で置く
    # キーの存在確認
    # p,n,e,?p?nの個数を数える
    for sentence in sentences:
        for token in tokenizer.tokenize(sentence):
            word = token.surface
            if word in np_dic:
                value = np_dic[word]
                if value in result:
                    result[value] += 1                        
    #総和を求める
    summary = result['p'] + result['n'] + result['e'] + result['?p?n']
    # ネガポジ度の平均（pの総数 / summary, nの総数 / summary）を数値でそれぞれ出力
    # summaryが0の場合もあるので、try-exceptで例外処理
    # ポジティブ度の平均
    # ネガティブ度の平均
    # summaryが0の場合
    try:
        print("ポジティブ度：",result['p'] / summary)
        print("ネガティブ度：",result['n'] / summary)
    except ZeroDivisionError:
        print("Summaryが0です")


# 実行部分
if __name__ == '__main__':
    main()

(0番目のつぶやき['ARAN@FE勉強中', 'aran_ai_python', 0, 27, '2021-01-29-23:56-44', 'おはようございます！#今日の積み上げ✅筋トレ\u3000足✅FE 午前ランダム✅FE 午後✅python 予想問題今日で丁度試験10日前になる。引き続き選択問題に多く触れたい。#駆け出しエンジニアと繋がりたい#プログラミング初心者']
(1番目のつぶやき['ペロ', 'tw_agmg', 0, 25, '2021-01-29-23:50-49', '16日目\u3000検定まで37日おはようございます。昨日は呟きませんでしたが、少しずつ積み上げております。今日も隙間に勉強します❗️#今日の積み上げ ✅pythonチュートリアル#Python3 #プログラミング独学 #プログラミング初心者と繋がりたい']
(2番目のつぶやき['ｓです', 'sho_privateblog', 0, 23, '2021-01-29-23:49-08', 'おはようございます！＃今日の積み上げ🍞Pythonの勉強🍞ヘッダー画像の作成🍞英語の勉強wordpressの無料テーマを使ってるのですが、あまり好きじゃないので少しづつカスタマイズしてきます！＃ブログ書け #ブログ仲間募集中']
(3番目のつぶやき['なおきち@筋肉系投資ブロガー', 'naokichipocket', 0, 13, '2021-01-29-23:39-07', 'おはようございます😃お酒を飲むと睡眠の質が下がりますね😅MyPF-1.71%クリエネ🔽ヘルスケア🔽ハイテク🔽半導体🔽見事にマイナスです笑チャアアアアアアン！(きんにくん)ToDo✅ブログ✅筋トレ✅Python✅料理✅左心耳について勉強今日はタフな飲み会になりそう\U0001f972#朝活 #今日の積み上げ']
(4番目のつぶやき['ryojiro@30代未経験→機械学習勉強中→データコンサルタントやりたい', 'MEF2101_dataC', 0, 10, '2021-01-29-23:35-27', '#今日の積み上げ昨日はDICの卒業発表を見学させていただきました！その後の懇親会も混ざらせていただきありがとうございます。だいぶお邪魔しました。今日はkaggle提出のスコアを上げながら、色々な手法を脳みそちゃんに